# Chopper Cascade

The following is for testing purposes.
We copy code from the `tof` docs, convert the choppers, and compare results:

In [ ]:
%matplotlib widget
from scippneutron.tof import chopper_cascade
import scipp as sc


def wavelength_to_inverse_velocity(wavelength):
    h = sc.constants.h
    m_n = sc.constants.m_n
    return (wavelength * m_n / h).to(unit='s/m')

In [ ]:
import tof

In [ ]:
Hz = sc.Unit('Hz')
deg = sc.Unit('deg')
meter = sc.Unit('m')
choppers = [
    tof.Chopper(
        frequency=70.0 * Hz,
        open=sc.array(
            dims=['cutout'],
            values=[98.71, 155.49, 208.26, 257.32, 302.91, 345.3],
            unit='deg',
        ),
        close=sc.array(
            dims=['cutout'],
            values=[109.7, 170.79, 227.56, 280.33, 329.37, 375.0],
            unit='deg',
        ),
        phase=47.10 * deg,
        distance=6.6 * meter,
        name="WFM1",
    ),
    tof.Chopper(
        frequency=70 * Hz,
        open=sc.array(
            dims=['cutout'],
            values=[80.04, 141.1, 197.88, 250.67, 299.73, 345.0],
            unit='deg',
        ),
        close=sc.array(
            dims=['cutout'],
            values=[91.03, 156.4, 217.18, 269.97, 322.74, 375.0],
            unit='deg',
        ),
        phase=76.76 * deg,
        distance=7.1 * meter,
        name="WFM2",
    ),
    tof.Chopper(
        frequency=56 * Hz,
        open=sc.array(
            dims=['cutout'],
            values=[74.6, 139.6, 194.3, 245.3, 294.8, 347.2],
            unit='deg',
        ),
        close=sc.array(
            dims=['cutout'],
            values=[95.2, 162.8, 216.1, 263.1, 310.5, 371.6],
            unit='deg',
        ),
        phase=62.40 * deg,
        distance=8.8 * meter,
        name="Frame-overlap 1",
    ),
    tof.Chopper(
        frequency=28 * Hz,
        open=sc.array(
            dims=['cutout'],
            values=[98.0, 154.0, 206.8, 254.0, 299.0, 344.65],
            unit='deg',
        ),
        close=sc.array(
            dims=['cutout'],
            values=[134.6, 190.06, 237.01, 280.88, 323.56, 373.76],
            unit='deg',
        ),
        phase=12.27 * deg,
        distance=15.9 * meter,
        name="Frame-overlap 2",
    ),
    tof.Chopper(
        frequency=7 * Hz,
        open=sc.array(
            dims=['cutout'],
            values=[30.0],
            unit='deg',
        ),
        close=sc.array(
            dims=['cutout'],
            values=[140.0],
            unit='deg',
        ),
        phase=0 * deg,
        distance=22 * meter,
        name="Pulse-overlap",
    ),
]

In [ ]:
source = tof.Source(facility='ess', neutrons=10_000_000)
source

In [ ]:
detectors = [
    tof.Detector(distance=26.0 * meter, name='monitor'),
    tof.Detector(distance=32.0 * meter, name='detector'),
]

In [ ]:
model = tof.Model(source=source, choppers=choppers, detectors=detectors)
model

In [ ]:
res = model.run()
res

In [ ]:
def from_tof_chopper(chopper: tof.Chopper) -> chopper_cascade.Chopper:
    """Convert a chopper from the tof package to a chopper for the chopper-cascade package."""
    return chopper_cascade.Chopper(
        distance=chopper.distance,
        time_open=chopper.open_close_times()[0].to(unit='s'),
        time_close=chopper.open_close_times()[1].to(unit='s'),
    )


choppers = [from_tof_chopper(chopper) for chopper in choppers]

In [ ]:
# Other examples (unused):
# pulse_overlap = chopper_cascade.Chopper(
#     distance=sc.scalar(22.0, unit='m'),
#     time_open=sc.scalar(0.0119, unit='s'),
#     time_close=sc.scalar(0.05555, unit='s'),
# )
#
# chopper = chopper_cascade.Chopper(
#     distance=sc.scalar(8.0, unit='m'),
#     time_open=sc.array(dims=['slit'], values=[0.0083333, 0.013888], unit='s'),
#     time_close=sc.array(dims=['slit'], values=[0.011111, 0.022222], unit='s'),
# )

time = sc.array(dims=['vertex'], values=[0.0, 5, 5, 0], unit='ms').to(unit='s')
wavelength = sc.array(dims=['vertex'], values=[0.01, 0.01, 10, 10], unit='angstrom')

frames = [
    chopper_cascade.Frame(
        distance=sc.scalar(0.0, unit='m'),
        subframes=[
            chopper_cascade.Subframe(
                time=time, inverse_velocity=wavelength_to_inverse_velocity(wavelength)
            )
        ],
    )
]

for chopper in choppers:
    frames.append(chopper_cascade.chop(frames[-1], chopper))
frames.append(chopper_cascade.propagate(frames[-1], distance=sc.scalar(26.0, unit='m')))

# chopper_cascade.draw_matplotlib(frames)

In [ ]:
fig2, ax2 = chopper_cascade.draw_matplotlib(frames)
fig1 = res.detectors['monitor'].tofs.plot(bins=2000)

# Extract line data from fig1
line1 = fig1.ax.lines[0]
xdata1, ydata1 = line1.get_data()
ax2.plot(xdata1 / 1e6, ydata1 > 0)
fig2.set_size_inches(14, 6)
fig2.tight_layout()

Note the agreement between the purple polygons and the blue line.